In [4]:
import argparse
from dataclasses import dataclass
import os
import shutil
import jax
import jax.numpy as jnp

from evojax.policy.base import PolicyNetwork, PolicyState
from evojax.task.base import TaskState
from evojax.task.slimevolley import SlimeVolley
from evojax.policy.mlp import MLPPolicy
from evojax.algo import CMA
from evojax import Trainer
from evojax import util

from flax import linen as nn

In [2]:
class NEATModel(nn.Module):
    input_dim: int
    output_dim: int
    max_nodes: int
    max_edges: int

    @nn.compact
    def __call__(self, x):
        nodes = self.param(
            "nodes", nn.initializers.zeros, (self.max_nodes, 3)
        )  # (id, activattion)
        edges = self.param(
            "edges", nn.initializers.zeros, (self.max_edges, 3)
        )  # (from, to, weight, enabled)
        values = jnp.zeros((self.max_nodes,))
        return jax.slice(values, -self.output_dim)


class NEATPolicy(PolicyNetwork):
    def __init__(self, input_dim: int, output_dim: int):
        self._input_dim = input_dim
        self._output_dim = output_dim
        # TODO: Implement NEAT policy.
        self.num_params = 1
        model = NEATModel(input_dim, output_dim, 100, 100)
        params = model.init(jax.random.PRNGKey(0), jnp.ones([1, input_dim]))
        print(params)
        self.num_params, format_params_fn = util.get_params_format_fn(params)
        self._format_params_fn = jax.vmap(format_params_fn)
        self._forward_fn = jax.vmap(model.apply)

    def get_actions(
        self, t_states, params, p_states: PolicyState
    ) -> tuple[jax.Array, PolicyState]:
        return self._forward_fn(params, t_states.obs), p_states

In [191]:
@jax.tree_util.register_dataclass
@dataclass
class Genome:
    """Stores node and connection info in JAX arrays."""

    node_ids: jax.Array  # shape [num_nodes]
    # 0: relu
    node_activation: jax.Array  # shape [num_nodes]

    # Connections.
    conn_id: jax.Array  # shape [num_connections]
    conn_in: jax.Array  # shape [num_connections]
    conn_out: jax.Array  # shape [num_connections]
    conn_weights: jax.Array  # shape [num_connections]
    conn_enabled: jax.Array  # shape [num_connections]


@dataclass(frozen=True)
class Config:
    """Stores static info for the NEAT algorithm."""

    input_dim: int
    output_dim: int
    max_nodes: int
    max_edges: int
    max_depth: int


config = Config(input_dim=2, output_dim=1, max_nodes=10, max_edges=20, max_depth=10)

In [186]:
node_ids = jnp.array([0, 1, 2, 3, 4])
node_activation = jnp.array([0, 0, 0, 0, 2])

# 0 1
# | |
# 2 3
#  \|
#   4
conn_id = jnp.array([0, 1, 2, 3])
conn_in = jnp.array([0, 1, 2, 3])
conn_out = jnp.array([2, 3, 4, 4])
conn_weights = jnp.array([1.0, 1.0, 1.0, 1.0])
conn_enabled = jnp.array([True, True, True, True])


def create_genome(
    config: Config,
    node_ids,
    node_activation,
    conn_id,
    conn_in,
    conn_out,
    conn_weights,
    conn_enabled,
) -> Genome:
    node_pad_config = [(0, config.max_nodes - node_ids.shape[0], 0)]
    node_ids = jax.lax.pad(node_ids, -1, node_pad_config)
    node_activation = jax.lax.pad(node_activation, -1, node_pad_config)

    conn_pad_config = [(0, config.max_edges - conn_id.shape[0], 0)]
    conn_id = jax.lax.pad(conn_id, -1, conn_pad_config)
    conn_in = jax.lax.pad(conn_in, -1, conn_pad_config)
    conn_out = jax.lax.pad(conn_out, -1, conn_pad_config)
    conn_weights = jax.lax.pad(conn_weights, jnp.nan, conn_pad_config)
    conn_enabled = jax.lax.pad(conn_enabled, False, conn_pad_config)
    return Genome(
        node_ids,
        node_activation,
        conn_id,
        conn_in,
        conn_out,
        conn_weights,
        conn_enabled,
    )


genome = create_genome(
    config,
    node_ids,
    node_activation,
    conn_id,
    conn_in,
    conn_out,
    conn_weights,
    conn_enabled,
)
genome = jax.tree.map(lambda x: jnp.array([x]), genome)
genome

Genome(node_ids=Array([[ 0,  1,  2,  3,  4, -1, -1, -1, -1, -1]], dtype=int32), node_activation=Array([[ 0,  0,  0,  0,  2, -1, -1, -1, -1, -1]], dtype=int32), conn_id=Array([[ 0,  1,  2,  3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1]], dtype=int32), conn_in=Array([[ 0,  1,  2,  3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1]], dtype=int32), conn_out=Array([[ 2,  3,  4,  4, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1]], dtype=int32), conn_weights=Array([[ 1.,  1.,  1.,  1., nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan]], dtype=float32), conn_enabled=Array([[ True,  True,  True,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False]], dtype=bool))

In [188]:
def get_adjacency_matrix_fn(config: Config, genome: Genome):
    matrix = jnp.zeros((config.max_nodes, config.max_nodes))
    matrix = matrix + jnp.pad(
        jnp.identity(config.input_dim),
        (
            (0, config.max_nodes - config.input_dim),
            (0, config.max_nodes - config.input_dim),
        ),
    )
    matrix = matrix.at[genome.conn_out, genome.conn_in].set(
        jnp.where(genome.conn_enabled, genome.conn_weights, 0.0)
    )
    return matrix


get_adjacency_matrix = jax.jit(
    jax.vmap(get_adjacency_matrix_fn, in_axes=(None, 0)), static_argnums=(0)
)

In [54]:
display(get_adjacency_matrix)

<PjitFunction of <function get_adjacency_matrix_fn at 0x3050f6840>>

In [196]:
adj = get_adjacency_matrix(config, genome)
display(adj)


def matmul_fn(adj, x):
    return jnp.matmul(adj, x)


matmul = jax.vmap(matmul_fn)
x = jnp.stack([jnp.pad(jnp.array([1, 1, 0, 0, 0]), (0, 5))])
display(matmul(adj, x))
display(matmul(adj, matmul(adj, x)))
display(matmul(adj, matmul(adj, matmul(adj, x))))

Array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)

Array([[1., 1., 1., 1., 0., 0., 0., 0., 0., 0.]], dtype=float32)

Array([[1., 1., 1., 1., 2., 0., 0., 0., 0., 0.]], dtype=float32)

Array([[1., 1., 1., 1., 2., 0., 0., 0., 0., 0.]], dtype=float32)

In [199]:
# TODO: Check NEAT-python for activation functions.
ACTIVATION_FUNCTIONS = [lambda x: x, jax.nn.relu, jax.nn.sigmoid, jax.nn.tanh]


def forward_fn(config: Config, genome: Genome, x: jax.Array):
    print(genome.node_ids.shape)
    adj = get_adjacency_matrix_fn(config, genome)
    for _ in range(config.max_depth):
        x = matmul_fn(adj, x)

        print(adj.shape)
        print(x.shape)
        print(genome.node_activation.shape)

        # Apply the activation function.
        def activate_fn(activation, x):
            return jax.lax.switch(activation, ACTIVATION_FUNCTIONS, x)

        activate = jax.vmap(activate_fn)
        x = activate(genome.node_activation, x)
    return x


forward = jax.jit(jax.vmap(forward_fn, in_axes=(None, 0, 0)), static_argnums=(0))
# forward = jax.vmap(forward_fn, in_axes=(None, 0, 0))

forward(config, genome, x)

(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)


Array([[1.      , 1.      , 1.      , 1.      , 0.880797, 0.      ,
        0.      , 0.      , 0.      , 0.      ]], dtype=float32)

In [203]:
genome2 = create_genome(
    config,
    node_ids,
    jnp.array([1, 1, 1, 1, 1]),
    conn_id,
    conn_in,
    conn_out,
    conn_weights,
    conn_enabled,
)
genome2 = jax.tree.map(lambda x: jnp.array([x]), genome2)

forward(config, genome2, x)

(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)
(10, 10)
(10,)
(10,)


Array([[1., 1., 1., 1., 2., 0., 0., 0., 0., 0.]], dtype=float32)

In [205]:
import neat

In [206]:
from evojax.task.base import TaskState

In [211]:
TaskState().__dict__

{}